## **Загрузка**

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Визуализация
import matplotlib.pyplot as plt

# Машинное обучение
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer

# Нейросети
import torch
import torchvision
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM, SpatialDropout1D

In [2]:
import tensorflow as tf

In [3]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/CSV

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/CSV


In [4]:
data = pd.read_csv('news.csv', usecols=range(1,3))

In [5]:
data.info(null_counts =1) 
print('\nКоличество повторяющихся строк —', data.duplicated().sum())
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19462 entries, 0 to 19461
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   news    19462 non-null  object 
 1   source  18360 non-null  float64
dtypes: float64(1), object(1)
memory usage: 304.2+ KB

Количество повторяющихся строк — 29


,news,source
0,Официальный аккаунт PlayStation опубликовал т...,1.0
1,Китайская компания Mobvoi опубликовала на офи...,1.0
2,Практически во всех странах мира введены огра...,0.0
3,"Депутат Госдумы, единоросс Антон Горелкин вне...",2.0
4,Совет директоров «Почты России» проголосовал ...,2.0


* Удалить дубликаты.
* Обработать текст для подачи на вход нейронной сети.

# **Предобработка**

## Удаление дубликатов

In [6]:
data_cleaned = data.drop_duplicates()

In [7]:
data_cleaned.duplicated().sum()

0

## Обработка текста

In [8]:
# Максимальное количество слов в новости
maxlen = max(data_cleaned.news.apply(lambda x: len(x.split())))
maxlen

6495

In [9]:
# Частота самого упоминаемого слова
max_features = data_cleaned.news.str.split(expand=True).stack().value_counts().max()
max_features

175854

In [10]:
tokenizer = Tokenizer(num_words=20000)

In [11]:
sentences = data_cleaned['news'].values

In [12]:
y = data_cleaned['source'].values

In [14]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

In [15]:
tokenizer.fit_on_texts(sentences_train)

In [16]:
X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

In [17]:
vocab_size = len(tokenizer.word_index) + 1

In [18]:
print(sentences_train[2])
print(X_train[2])

 Стриминговый сервис Spotify в настоящее время испытывает проблемы в работе: пользователи жалуются, что не загруженные в офлайн треки воспроизводятся несколько секунд, после чего приостанавливаются. Сервис Downdetector, фиксирующий сбои сервисов, передаёт о почти 6000 отчётах о перебоях в работе Spotify. Судя по карте, чаще всего пользователи регистрируют недоступность стриминговой платформы в Европе и США, но есть отчёты и из Африки, Австралии, Южной Америки. Есть проблемы и у российских клиентов. В Twitter-профиле Spotify Status появилось сообщение следующего содержания: «Мы в курсе некоторых проблем, наблюдаемых в настоящее время, и проверяем их». Spotify официально появился в России и ряде других постсоветских стран в середине июля. 
[4088, 201, 1407, 1, 1449, 69, 11462, 468, 1, 584, 143, 3857, 5, 6, 14002, 1, 4270, 7524, 117, 3263, 66, 453, 201, 5472, 5420, 366, 17641, 15, 346, 5719, 15, 1, 584, 1407, 1547, 8, 3939, 1049, 103, 143, 15605, 539, 1, 1034, 2, 72, 18, 65, 16091, 2, 10,

In [19]:
print(sentences_test[2])
print(X_test[2])

 Испанский парламент принял новый закон об интеллектуальной собственности, обязывающий агрегаторы новостей платить издателям за контент, сообщила сегодня испанская газета El Pais. Интернет-компании должны будут делать это всякий раз, когда в результатах поиска агрегатора появляется та или иная новость. За принятие этого закона, по данным El Pais, проголосовали 172 депутата нижней палаты парламента, против были 144 депутата, трое воздержались. Закон начнет действовать с 1 января 2015 г. Однако размер компенсации в нем не указан.  Правительство Испании надеется, что этот закон ослабит интернет-пиратство, улучшит систему управления авторскими правами и заставит такие агрегаторы новостей, как Google News, компенсировать СМИ ущерб от использования их контента, отмечает газета.  Многие называют этот закон «налогом на Google», который является самым популярным поисковиком в Испании. По данным StatCounter, его доля на конец сентября составляла 95,3%. В Испании эту инициативу чаще называют AEDE

In [20]:
for word in ['в', 'об', 'сервис', 'парламент']:
...     print('{}: {}'.format(word, tokenizer.word_index[word]))

в: 1
об: 85
сервис: 201
парламент: 10873


In [21]:
data_cleaned['news_numeric'] = tokenizer.texts_to_sequences(data_cleaned.news)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [22]:
data_cleaned.head()

,news,source,news_numeric
0,Официальный аккаунт PlayStation опубликовал т...,1.0,"[1052, 1137, 734, 674, 2384, 1077, 202, 122, 3..."
1,Китайская компания Mobvoi опубликовала на офи...,1.0,"[2133, 35, 17215, 1771, 3, 697, 285, 3389, 136..."
2,Практически во всех странах мира введены огра...,0.0,"[418, 74, 134, 588, 562, 10341, 882, 1822, 4, ..."
3,"Депутат Госдумы, единоросс Антон Горелкин вне...",2.0,"[3323, 2260, 2726, 8254, 7043, 1, 13557, 7598,..."
4,Совет директоров «Почты России» проголосовал ...,2.0,"[1515, 885, 4473, 1257, 12, 9075, 7200, 3433, ..."


In [23]:
data_cleaned['news_numeric'] = data_cleaned['news_numeric'].apply(lambda x: np.array(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# **Расчёты**

In [24]:
#  Необходимое количество предсказаний
len(data_cleaned.query('source != source'))

1102

In [25]:
X = data_cleaned.query('source == source')['news_numeric']
y = data_cleaned.query('source == source')['source']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

In [49]:
X_train

array([[    0,     0,     0, ...,  4543,   879,   914],
       [    0,     0,     0, ...,   105,    32,   447],
       [    0,     0,     0, ...,  1009,  5201,  4512],
       ...,
       [    0,     0,     0, ..., 12628,  2577,     1],
       [    0,     0,     0, ...,  3028, 11361,  6655],
       [    0,     0,     0, ..., 11007,     2, 19527]], dtype=int32)

In [27]:
X_train = [torch.from_numpy(X_train[x]) for x in X_train.index]

In [28]:
X_test = [torch.from_numpy(X_test[x]) for x in X_test.index]

In [29]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [30]:
y_train = tf.stack(y_train)

In [31]:
y_test = tf.stack(y_test)

In [32]:
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2)) 
model.add(Dense(1, activation="sigmoid"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [33]:
len(X_train[0])

6495

In [34]:
y_train[0]

<tf.Tensor: shape=(), dtype=float64, numpy=1.0>

In [35]:
model.fit(X_train, y_train, batch_size=64, epochs=6,
          validation_data=(X_test, y_test), verbose=2)

Epoch 1/6
230/230 - 4492s - loss: -4.8146e-02 - accuracy: 0.6434 - val_loss: -2.5022e-01 - val_accuracy: 0.6370
Epoch 2/6
230/230 - 4416s - loss: -2.3521e+00 - accuracy: 0.7014 - val_loss: -5.2132e+00 - val_accuracy: 0.7499
Epoch 3/6
230/230 - 4370s - loss: -7.1317e+00 - accuracy: 0.7327 - val_loss: -8.6142e+00 - val_accuracy: 0.7497
Epoch 4/6
230/230 - 4428s - loss: -7.8700e+00 - accuracy: 0.7186 - val_loss: -7.2698e+00 - val_accuracy: 0.7096
Epoch 5/6
230/230 - 4594s - loss: -1.3638e+01 - accuracy: 0.7767 - val_loss: -1.0184e+01 - val_accuracy: 0.7128
Epoch 6/6
230/230 - 4649s - loss: -1.7767e+01 - accuracy: 0.7776 - val_loss: -1.5980e+01 - val_accuracy: 0.7423


In [124]:
test = data_cleaned.query('source != source')

In [119]:
test.head()

,news,source,news_numeric
18360,Глава форума Reddit Стив Хаффман раскритикова...,NaN,"[336, 3536, 2454, 2789, 6877, 680, 10525, 920,..."
18361,Французское архитектурное бюро Vincent Calleb...,NaN,"[5180, 6429, 295, 8, 1, 9759, 5059, 295, 410, ..."
18362,"Samsung провела мероприятие Galaxy Unpacked, ...",NaN,"[104, 2202, 1772, 112, 4504, 1, 252, 291, 118,..."
18363,В бета-версии клиента Telegram для Android с ...,NaN,"[1, 1005, 177, 2931, 164, 9, 77, 4, 16587, 37,..."
18364,Журналистам из XDA удалось раздобыть минималь...,NaN,"[3553, 10, 3166, 447, 11762, 9159, 963, 77, 17..."


In [95]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1102 entries, 18360 to 19461
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   news          1102 non-null   object 
 1   source        0 non-null      float64
 2   news_numeric  1102 non-null   object 
dtypes: float64(1), object(2)
memory usage: 34.4+ KB


In [125]:
test_news_numeric =  test[['news_numeric']].to_numpy()

In [127]:
test_news_numeric = [torch.from_numpy(test['news_numeric'][x]) for x in (test.index)]

In [129]:
test['source'] = tf.stack(test['source'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [132]:
test_news_numeric = sequence.pad_sequences(test_news_numeric, maxlen=maxlen)

In [133]:
prediction = model.predict(test_news_numeric)

___

# **Выгрузка**

In [137]:
len(prediction)

1102

In [145]:
rt = pd.Series(prediction.ravel())

In [146]:
rt.head()

0    0.014508
1    1.000000
2    1.000000
3    1.000000
4    1.000000
dtype: float32

In [151]:
rt = np.floor(rt)

In [152]:
rt.to_csv('news_matveeva-a.csv', index=False, header=False)